In [49]:
mp_pdf = True
import sys, os
import django
if mp_pdf:
    extension = "pdf"
    import matplotlib
    matplotlib.use('pgf')
    pgf_with_latex = {
        "text.usetex": True,            # use LaTeX to write all text
        "pgf.rcfonts": False,           # Ignore Matplotlibrc
        "text.latex.unicode": True,
        "pgf.preamble": [
            #r"\usepackage[utf8x]{inputenc}",
            r"\usepackage{xcolor}"
        ],
        "pgf.texsystem" : "xelatex",
        "figure.figsize": [12,7]
    }
    matplotlib.rcParams.update(pgf_with_latex)
else:
    extension = "png"
    
import matplotlib as mpl

sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *


dsize = 7.0866142
run_id=1861
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from adjustText import adjust_text, get_renderer, get_bboxes
import operator

mpl.rcParams['font.size'] = 6

In [23]:
mdf = pd.read_csv(f'../tables/newness_representation_{run_id}.csv')
mdf.head()

mdf[mdf['primary_wg']==3].head()

,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,lrep,x_q,y_q
1,0.143624,17.853955,0.004963,3,199.901053,0.005426,"sector, sectoral emissions",-0.000463,0.000463,0.914610,0.542141,4.593220,-0.089258,7,5
6,0.068143,6.868519,0.001909,3,169.577758,0.004603,"electricity, electricity generation & demand",-0.002694,0.002694,0.414773,0.535850,4.620690,-0.880023,7,1
14,0.086559,16.772689,0.004662,3,323.258870,0.008775,"technology, technology developement & carbon c...",-0.004112,0.004112,0.531335,0.536115,4.620690,-0.632362,7,2
22,0.183268,39.718230,0.011040,3,343.127380,0.009314,"countries, country analysis and comparisons",0.001726,0.001726,1.185362,0.530242,4.508475,0.170048,5,7
23,0.137934,22.423120,0.006233,3,263.391556,0.007150,"prices, (CO2) prices and markets",-0.000917,0.000917,0.871788,0.535281,4.603448,-0.137209,7,5


In [24]:
topic_dis_df = pd.read_csv(f'../tables/topic_discipline_{run_id}.csv')
topic_dis_df = topic_dis_df.rename(columns={"dis_share":"dis_score"})

topic_dis_df['dis_total'] = topic_dis_df.groupby('title')['dis_score'].transform('sum')

topic_dis_df['dis_share'] = topic_dis_df['dis_score'] / topic_dis_df['dis_total']

topic_dis_df.head()

,dis_score,doctopic__doc__wc__oecd,primary_wg,title,dis_share_pre,dis_share_post,dis_total,dis_share
0,98.609781,Agricultural Sciences,2,"indices, indices, correlation and relationships",45.168047,53.441733,1028.065952,0.095918
1,185.606297,Engineering and Technology,2,"indices, indices, correlation and relationships",73.102808,112.503490,1028.065952,0.180539
2,2.184284,Humanities,2,"indices, indices, correlation and relationships",0.756985,1.427299,1028.065952,0.002125
3,13.903700,Medical and Health Sciences,2,"indices, indices, correlation and relationships",5.620609,8.283041,1028.065952,0.013524
4,655.278080,Natural Sciences,2,"indices, indices, correlation and relationships",310.473621,344.804458,1028.065952,0.637389


In [25]:
topic_dis_wide = topic_dis_df.pivot_table(
    index=["title"],columns="doctopic__doc__wc__oecd",values="dis_share"
).reset_index()

topic_dis_wide.head()

doctopic__doc__wc__oecd,title,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences
0,"(e)-vehicles, electric, hybrid and combustion ...",0.017059,0.569020,0.002428,0.006232,0.247428,0.154439
1,"CH4, CH4",0.204914,0.166121,0.000209,0.003133,0.618849,0.006607
2,"CLC, chemical looping combustion (CLC)",0.035575,0.512716,0.001255,0.004690,0.426735,0.017448
3,"CO2 storage, geological CO2 storage",0.047830,0.484719,0.001622,0.004390,0.434192,0.026446
4,"CO2, CO2",0.084190,0.363998,0.000726,0.002875,0.527574,0.019829


In [26]:
df = pd.concat([mdf.sort_values('title').reset_index(drop=True),topic_dis_wide.sort_values('title').reset_index(drop=True)],axis=1)
df = pd.merge(mdf, topic_dis_wide)
df.head()

,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,representation,...,year_av,lrep,x_q,y_q,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences
0,0.127550,16.243344,0.004515,1,191.274690,0.005192,"transport, transport",-0.000677,0.000677,0.869630,...,4.446154,-0.139688,3,5,0.034290,0.286544,0.002646,0.008167,0.527715,0.138370
1,0.143624,17.853955,0.004963,3,199.901053,0.005426,"sector, sectoral emissions",-0.000463,0.000463,0.914610,...,4.593220,-0.089258,7,5,0.055740,0.393644,0.005167,0.007054,0.325992,0.208360
2,0.275522,44.160043,0.012275,1,207.308023,0.005627,"uncertainty, uncertainty & estimation",0.006648,0.006648,2.181373,...,4.521739,0.779954,5,9,0.069837,0.200647,0.004018,0.007810,0.632479,0.084286
3,0.127781,22.153958,0.006158,1,224.378004,0.006091,"air pollution, air pollution",0.000068,0.000068,1.011085,...,4.402778,0.011024,2,6,0.072704,0.295369,0.003491,0.019946,0.548198,0.058452
4,0.143087,26.783303,0.007445,2,231.589446,0.006286,"humans, human activity, human behaviour & impa...",0.001159,0.001159,1.184300,...,4.527027,0.169152,5,7,0.065904,0.155102,0.017031,0.027815,0.621609,0.111205


In [27]:
mdf['elrep'] = np.exp(abs(mdf['lrep']))
mdf[mdf['title'].str.contains("biochar|adsorption|building",regex=True)]


,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,lrep,x_q,y_q,elrep
8,0.043920,3.519250,0.000978,2,147.331131,0.003999,"adsorption, adsorption (of CO2 for capture)",-0.003021,0.003021,0.244609,0.526849,4.714286,-1.408095,9,0,4.088160
34,0.065839,2.339483,0.000650,3,86.528908,0.002349,"biochar, biochar",-0.001698,0.001698,0.276870,0.436873,4.702128,-1.284209,9,0,3.611810
51,0.056965,4.560372,0.001268,2,156.163661,0.004239,"buildings, buildings",-0.002971,0.002971,0.299045,0.495174,4.716981,-1.207161,9,0,3.343976


In [28]:
mdf[mdf['title'].str.contains("food|health|adaptation",regex=True)]

,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,lrep,x_q,y_q,elrep
25,0.186513,20.538360,0.005709,2,205.288835,0.005572,"health, public health impacts",0.000137,0.000137,1.024512,0.475751,4.686275,0.024216,9,6,1.024512
42,0.353378,41.059011,0.011413,2,189.776405,0.005151,"adaptation, adaptation & vulnerability",0.006262,0.006262,2.215556,0.511459,4.722222,0.795504,9,9,2.215556
69,0.140790,13.314029,0.003701,2,167.337119,0.004542,"food, food",-0.000841,0.000841,0.814768,0.512890,4.672727,-0.204852,9,4,1.227344


In [99]:
pdf = df[df['primary_wg']==3]

fig, ax = plt.subplots(figsize=(dsize*0.5,dsize*0.3), dpi=200)

def set_log_ticks(fdf, ax, min_y=-8, tight=False):
    fmax = None
    fmin = None
    rmax = fdf['lrep'].max()
    if fmax:
        rmax = fmax
    rmin = fdf['lrep'].min()
    if fmin:
        rmin = -fmin

    if rmin > -1:
        rmin = -1
    if np.exp(rmax) < 2:
        rmax = np.log(2.1)

    lmin = int(1/np.exp(rmin)//1)*-1
    lmax = int(np.exp(rmax)//1)

    all_ytick_labels = [i for i in range(lmin,lmax+1) if i not in [-1,1]]

    ytick_labels = []

    if tight:
        for x in all_ytick_labels:
            if lmin > min_y:
                ytick_labels.append(x)   
            elif abs(x) in [2,10,50]:
                ytick_labels.append(x)
    else:     
        for x in all_ytick_labels:
            if lmin > min_y:
                ytick_labels.append(x)
            else:
                if abs(x) < 11:
                    if x % 5 == 0 or abs(x)==2:
                        ytick_labels.append(x)
                elif x < 100:
                    if x % 50==0 or abs(x)==20:
                        ytick_labels.append(x)


    yticks = []
    ytick_label_fractions = []
    for x in ytick_labels:
        if x < 0:
            y = np.log(-1/x)
            ytick_label_fractions.append('$\\frac{1}{'+str(abs(x))+'}$')
        elif x==0:
            y = 0
            ytick_label_fractions.append(1)
        else:
            y = np.log(x)
            ytick_label_fractions.append(x)
        yticks.append(y)

    ax.set_yticks(yticks)
    ax.set_yticklabels(ytick_label_fractions)   

def wg_socsci_rep(ax, pdf, c):
    ax.scatter(
        pdf['Social Sciences'], 
        pdf['lrep'],
        c=c,
        linewidths=0.5,
        edgecolors="grey",
        s=5
    )
    texts = []
    for i, row in pdf.iterrows():
        stext = row['title'].split(',')[0].replace('{','')
        texts.append(ax.text(row['Social Sciences'], row['lrep'], stext, fontsize=6))

    ax.axhline(0,c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)
    ax.set_xlabel("Social science proportion")
    ax.set_ylabel("IPCC representation")
    
    set_log_ticks(pdf, ax)

    adjust_text(texts, ax=ax)
    
wg_socsci_rep(ax, pdf, "#8da0cb")

fig.patch.set_facecolor('white') 

#fig.tight_layout()

plt.savefig(
    f'../plots/run_{run_id}_wg3_socsci.{extension}',
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)
plt.show()

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [103]:
fig, axs = plt.subplots(3,figsize=(dsize,dsize),dpi=200)
colors=["#66c2a5","#fc8d62","#8da0cb"]
for wg in [1,2,3]:
    
    ax = axs[wg-1]

    pdf = df[df['primary_wg']==wg]
    c = colors[wg-1]
    wg_socsci_rep(ax,pdf,c)
    
    set_log_ticks(pdf, ax,-4)
    
    
fig.tight_layout()

plt.savefig(
    f'../plots_pub/wgs_socsci.{extension}',
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [31]:
fig, ax = plt.subplots(figsize=(dsize*0.5,dsize*0.3),dpi=200)
import textwrap as tw


oecd_df_ys = pd.read_csv(f'../tables/oecd_representation_{run_id}.csv')
fdf = oecd_df_ys[oecd_df_ys['ar']>2]

fdf['ar_share_total'] = fdf.groupby('ar')['share'].transform('sum')



size = 25

cdict = {
    'Natural Sciences': '#0072B2',
    'Engineering and Technology': '#CC79A7',
    'Agricultural Sciences': '#009E73',
    'Social Sciences': '#E69F00',
    'Medical and Health Sciences': '#F0E442',
    'Humanities': '#000000',
    None: '#F0F0F0'
}

 

def simplified_oecd_rep(ax, fdf, g="wc__oecd", fs = 5.8, w = 13, log_ticks=True):
    ticks = []
    labels = []
    
    w = 13
    
    fdf = fdf.sort_values('total_share').replace(-np.inf, np.nan).dropna()

    for i, (name, group) in enumerate(fdf.sort_values('total_share').replace(-np.inf, np.nan).dropna().groupby(g, sort=False)):
        group = group.reset_index(drop=True)
        try:
            color = cdict[name]
        except:
            color = cdict[group['oecd'].iloc[0]]
        group = group.replace(-np.inf, np.nan).dropna().sort_values('ar').reset_index(drop=True)
        if group.shape[0]==0:
            continue
        name = f"{name} ({group['total_share'].iloc[0]:.1%})"
        name = name.replace(' and ',' \& ')
        name = tw.fill(tw.dedent(name),width=w)
        ticks.append(i)
        labels.append(name)
        #color = group.reset_index()['color'][0]
        
        group['s_share'] = group['share'] / group['ar_share_total']

        start = group['lrep'][0]
        end = group['lrep'][len(group)-1]
        ax.plot(
            [i,i],
            [start,end],
            color="grey",
            zorder=-50,
            lw=0.5
        )
        ax.scatter(
            i,
            start,
            label=None,
            color=color,
            s = size,
            edgecolor="grey",
            lw=0.5
            #marker="x"
        )
        ax.scatter(
            i,
            end,
            label=None,
            color=color,
            s = size,
            edgecolor="grey",
            marker="^",
            lw=0.5
        )


    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, ha="center", va="top", rotation=0, fontsize=fs)
    #ax.set_xticklabels(labels, ha="right", va="top", rotation=25)
    ax.axhline(0,c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)
    
    if log_ticks:
        set_log_ticks(fdf, ax)

    for l in [("AR3","o"),("AR5","^")]:
        ax.scatter(
            [],[],
            c='#f0f0f0',s=size,
            edgecolor="grey",
            label=l[0],marker=l[1],
            lw=0.5
        )

    ax.legend()
    
    ax.set_xlabel("Discipline")
    ax.set_ylabel("IPCC representation)")
    
    
simplified_oecd_rep(ax, fdf)

fig.patch.set_facecolor('white') 


plt.savefig(
    '../plots/ipcc_representation/ipcc_rep_oecds_simplified.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)


plt.show()

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [32]:
oecd_df_ys['elrep'] = np.exp(abs(oecd_df_ys['lrep']))
oecd_df_ys[oecd_df_ys['ar']==5]

,ar,color,ipcc_share,share,total_share,wc__oecd,representation,lrep,ar_share_total,elrep
28,5,#7fc97f,0.046350,0.105460,0.102866,Agricultural Sciences,0.439501,-0.822115,1.167779,2.275307
29,5,#beaed4,0.079091,0.276170,0.260545,Engineering and Technology,0.286384,-1.250421,1.167779,3.491813
30,5,#fdc086,0.001921,0.008823,0.009045,Humanities,0.217752,-1.524400,1.167779,4.592385
31,5,#ffff99,0.022654,0.022348,0.018990,Medical and Health Sciences,1.013728,0.013635,1.167779,1.013728
32,5,#386cb0,0.765610,0.651200,0.658729,Natural Sciences,1.175690,0.161856,1.167779,1.175690
33,5,#f0027f,0.136567,0.103310,0.101383,Social Sciences,1.321922,0.279087,1.167779,1.321922
34,5,#F0F0F0,0.000000,0.000469,0.001772,NaN,0.000000,-inf,1.167779,inf


In [101]:
from string import ascii_lowercase
wc_df_ys = pd.read_csv(f'../tables/wc_representation_{run_id}.csv')
wc_df_ys.head()

fdf = wc_df_ys[wc_df_ys['ar']>2]

#fdf = fdf[fdf['oecd']=="Agricultural Sciences"]

fdf['ar_share_total'] = fdf.groupby('ar')['share'].transform('sum')

#fig, axs = plt.subplots(3,2,figsize=(dsize,dsize),dpi=200)

fig = plt.figure(figsize=(dsize,dsize),dpi=200)

lax = None

y1 = 0
y2 = 0

gsize = (3,5)

axs = [
    [
        plt.subplot2grid(gsize, (0, 0), colspan=2, rowspan=1),
        plt.subplot2grid(gsize, (1, 0), colspan=2, rowspan=1),
        plt.subplot2grid(gsize, (2, 0), colspan=2, rowspan=1)
    ],
    [
        plt.subplot2grid(gsize, (0, 2), colspan=3, rowspan=1),
        plt.subplot2grid(gsize, (1, 2), colspan=3, rowspan=1),
        plt.subplot2grid(gsize, (2, 2), colspan=3, rowspan=1)
    ],
]

ids = [
    ['a','c','e'],
    ['b','d','f']
]

for i, (name, fdf) in enumerate(fdf.groupby('oecd')):
            
    if len(fdf['oecd_fos_text'].unique()) < 6:
        col = 0
        y = y1
        y1+=1
    else:
        col = 1
        y = y2
        y2+=1
        
    ax = axs[col][y]

    simplified_oecd_rep(ax, fdf, "oecd_fos_text", 4.4, log_ticks=True)
    ax.set_title(f"{ids[col][y]}.) {name} subfield representation", loc="left", fontweight="bold")
    set_log_ticks(fdf.sort_values('total_share').replace(-np.inf, np.nan).dropna(), ax, 5, True)

  
axs = axs[0] + axs[1]
ymin = min(a.get_ylim()[0] for a in axs)
ymax = max(a.get_ylim()[1] for a in axs)

ylab = 0

for a in axs:
    if a.get_yticks()[0] < ylab:
        ylab = a.get_yticks()[0]
        yticks = a.get_yticks()
        yticklabels = a.get_yticklabels()
    a.set_ylim((ymin,ymax))
        
for a in axs:
    a.set_yticks(yticks)
    a.set_yticklabels(yticklabels)
    
    
fig.tight_layout()
fig.patch.set_facecolor('white') 

plt.savefig(
    '../plots_pub/ipcc_rep_wcs_simplified.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()



/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [34]:
fdf = oecd_df_ys[oecd_df_ys['ar']>2]

fdf['ar_share_total'] = fdf.groupby('ar')['share'].transform('sum')
#fig, axs = plt.subplots(2,1,figsize=(dsize*0.5,dsize*0.7), dpi=200)
#fig, axs = plt.subplots(1,2,figsize=(dsize*0.5,dsize*0.7))
fig, axs = plt.subplots(1,2,figsize=(dsize,dsize*0.4))#,sharey=True)


simplified_oecd_rep(axs[0], fdf)

wg_socsci_rep(axs[1], pdf, "#8da0cb")

axs[1].set_ylabel(None)
axs[1].set_yticks([])

ymin = min(a.get_ylim()[0] for a in axs)
ymax = max(a.get_ylim()[1] for a in axs)

for a in axs:
    a.set_ylim((ymin,ymax))

#axs[0].set_ylim(axs[1].get_ylim())

fig.patch.set_facecolor('white') 
fig.tight_layout()

plt.savefig(
    '../plots_pub/panel_representation.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
def rep_time(ax, mdf, lquads=False, log_ticks=False, fs=7, quantiles=10):
    x = 'year_av'
    y = 'representation'
    y = 'lrep'

    mdf['lrep'] = np.log(mdf['representation'])

    cmap = {1: "#66c2a5", 2: "#fc8d62" , 3: "#8da0cb"}
    colors = [cmap[i] for i in mdf['primary_wg']]


    mdf.plot.scatter(
        x,y,s=mdf['score']*0.03,
        c = colors,ax=ax,
        lw=0.5,
        edgecolor="grey"
    )
    
    for k,v in cmap.items():
        ax.scatter(
            [],[],
            c=v,s=size,
            edgecolor="grey",
            lw=0.5,
            label=f"WG {k}",
        )   
        
    


    # Fit a trendline
    #z = np.polyfit(x=mdf[x], y=mdf[y], deg=1)
    #p = np.poly1d(z)
    #mdf['trendline'] = p(mdf.loc[:, x])
    #plt.plot(mdf[x],mdf['trendline'])


    # Quantile x and y
    qs = quantiles
    mdf['x_q'] = pd.qcut(mdf[x], qs, labels=False)
    mdf['y_q'] = pd.qcut(mdf[y], qs, labels=False)
    texts = []
    
    lfs = fs - 1
    if lfs < 5:
        lfs=5

    # write texts for dots in outer quantiles
    for i, row in mdf[(mdf['x_q'].isin([0,qs-1])) | (mdf['y_q'].isin([0,qs-1]))].iterrows(): 
        t = row['title'].split(',')[0].strip('{} ')
        texts.append(
            ax.text(row[x],row[y],t,ha='center', va='center',fontsize=lfs)
        )


    # Split the graph into quadrants
    ymid = 0
    ax.axhline(ymid,c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)
    ax.axvline(mdf[x].median(),c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)



    # Put labels on the quadrants
    # x
    d1 = mdf[x].median()-plt.xlim()[0]
    d2 = mdf[x].median()-plt.xlim()[1]

    xlabpoints = [mdf[x].median()-d1/2,mdf[x].median()-d2/2]
    
    xrange = mdf[x].max()-mdf[x].min()
    xlabpoints = [mdf[x].min()+xrange*0.15,mdf[x].min()+xrange*0.85]
    
    pad = (mdf[x].min() - ax.get_xlim()[0])*0.25
    
    xlabpoints = [ax.get_xlim()[0] + pad, mdf[x].median() + pad]
    

    
    if lquads:
        labels = [
            ['Under-represented,\nolder topics','Under-represented,\nnewer topics'],
            ['Over-represented,\nolder topics','Over-represented,\nnewer topics']
        ]

        pylims = ax.get_ylim()

        tpad = (pylims[1]-pylims[0])*0.13
        
        tpad=-pad*12

        ops = [operator.lt,operator.gt]

        for i in [0,1]:
            for j in [0,1]:
                if j==0:
                    tpadx=tpad*-1
                else:
                    tpadx=tpad*1


                q = mdf[(ops[i](mdf[x],mdf[x].median())) & (ops[j](mdf[y],ymid))]
                qshare = q['score'].sum()/mdf['score'].sum()
                l = labels[j][i].replace("\n"," ")
                ax.text(
                    xlabpoints[i],
                    pylims[j]+tpadx,
                    f"{l} {qshare:.0%}",
                    va="center",ha="left",
                    fontweight='bold',
                    #bbox={'facecolor':'red', 'alpha':qshare*2-0.1, 'pad':6},
                    fontsize=fs
                )

    if log_ticks:
        set_log_ticks(mdf, ax)

    #ax.get_yaxis().set_visible(False)
    #ax.tick_params(axis=u'both', which=u'both',length=0)
    #ax2.tick_params(axis=u'both', which=u'both',length=0)
    ax.set_xlabel('Assessment period\noccurence')
    ax.set_ylabel('IPCC representation')
    ax.legend()
    
    # Tweak the labels to avoid overlap    
    adjust_text(
        texts, 
        arrowprops=dict(arrowstyle="-", color='black', lw=0.5)
    )

fig, ax = plt.subplots(figsize=(dsize,dsize*0.62))

rep_time(ax, mdf, lquads=True, log_ticks=True)

plt.tight_layout(h_pad=100)

fig.patch.set_facecolor('white')    


plt.savefig(
    '../plots_pub/rep_time.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

In [96]:
fig = plt.figure(figsize=(dsize, dsize*1.1), dpi=200)


gsize = (8,2)

ax1 = plt.subplot2grid(gsize, (0, 0), colspan=1, rowspan=3)

ax2 = plt.subplot2grid(gsize, (0, 1), colspan=1, rowspan=3)

simplified_oecd_rep(ax1, fdf)

wg_socsci_rep(ax2, pdf, "#8da0cb")

ax2.set_ylabel("")
ax2.set_yticks([])

axs = [ax1,ax2]

ymin = min(a.get_ylim()[0] for a in axs)
ymax = max(a.get_ylim()[1] for a in axs)

for a in axs:
    a.set_ylim((ymin,ymax))

ax3 = plt.subplot2grid(gsize, (3, 0), colspan=2, rowspan=5)

ax1.set_title(
    "a) IPCC representation of disciplines", 
    fontsize=7, fontweight="bold",loc="left"
)

ax2.set_title(
    "b) Social science proportion and IPCC representation of WG 3 topics", 
    fontsize=7, fontweight="bold",loc="left"
)

ax3.set_title(
    "c) Topic novelty and representation", 
    fontsize=7, fontweight="bold",loc="left"
)

rep_time(ax3, mdf, lquads=True, log_ticks=True)

fig.canvas.draw()

fig.tight_layout()

fig.patch.set_facecolor('white')    

plt.savefig(
    '../plots_pub/big_panel_representation.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [16]:
import math


tdf = pd.read_csv(f'../tables/topic_info_{run_id}.csv')

tdf = pd.merge(df, tdf)

tdf = df

tdf['H'] = 0
for k,v in cdict.items():
    if k is None:
        continue
    tdf['H'] += tdf[k]+np.log(tdf[k])

tdf['H'] = -tdf['H']

n = 4

y = "H"

tdf = tdf.sort_values(y).reset_index(drop=True)
tdf = tdf.reset_index()

fig, axs = plt.subplots(
    1,n,
    figsize=(dsize,dsize*0.7),dpi=200,
    sharex=True
)

pdfs = []
chunks = math.ceil(tdf.shape[0]/n)
    
for i in range(n):
    f = i*chunks
    l = (i+1)*chunks
    pdfs.append(tdf[f:l].sort_values(y,ascending=False).reset_index(drop=True))

for pdf, ax in zip(pdfs,axs):
    b = 0
    for k,v in cdict.items():
        if k is None:
            continue
        s = pdf[k]
        
        ax.barh(
            pdf.index,
            s,
            #color = [colors[x-1] for x in pdf['primary_wg']],
            color=v,
            left=b,
            edgecolor='grey',
            linewidth=0.5,
            alpha=0.7,
            zorder=-10,
            label=k
        )
        
        b+=s
        
    ax.scatter(
        pdf[y]/tdf[y].max(),
        pdf.index,
        marker="x",
        s=10,
        c="grey",
        label="Disciplinary Entropy"
    )
    ax.plot(
        pdf[y]/tdf[y].max(),
        pdf.index,
        lw=0.5,
        c="grey"
    )

    ax.set_yticks(pdf.index)
    ax.set_yticklabels([f"{x['index']}. {x['title'].split(',')[0].replace(' ',' ')}" for i, x in pdf.iterrows()],fontsize=5)
    
    ax.set_xticks([0,1])
    
    ax.set_xticklabels([0,round(tdf[y].max(),2)],fontsize=6)

    ax.set_ylim(-1,math.ceil(tdf.shape[0]/n))
    
fig.tight_layout()

fig.patch.set_facecolor('white')    

#axs[0].legend(bbox_to_anchor=(0., 1.02, n+n-1, .102), loc=3,
       #ncol=2, mode="expand", borderaxespad=0.,fontsize=5)

s = fig.subplotpars
bb=[s.left, s.top+0.02, s.right-s.left, 0.05 ]

handles, labels = ax.get_legend_handles_labels()

handles = handles[1:2] + handles[:1] + handles[2:]
labels = labels[1:2] + labels[:1] + labels[2:]

leg = axs[0].legend(
    handles, labels,
    loc=8,bbox_to_anchor=bb, ncol= 6, mode="expand", borderaxespad=0,
    bbox_transform=fig.transFigure, fancybox=False, edgecolor="k",fontsize=5
)





plt.savefig(
    '../plots_pub/topic_oecd_entropy.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

In [17]:
tdf.head()

,index,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,...,lrep,x_q,y_q,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences,H
0,0,0.186513,20.538360,0.005709,2,205.288835,0.005572,"health, public health impacts",0.000137,0.000137,...,0.024216,9,6,0.063461,0.154936,0.010308,0.219381,0.435788,0.114000,12.718165
1,1,0.120808,30.419014,0.008456,2,328.077036,0.008905,"research, research agenda & sustainability sci...",-0.000450,0.000450,...,-0.051841,7,5,0.059428,0.212919,0.024153,0.020173,0.435313,0.244880,13.238373
2,2,0.140790,13.314029,0.003701,2,167.337119,0.004542,"food, food",-0.000841,0.000841,...,-0.204852,9,4,0.164710,0.173930,0.007289,0.043704,0.499010,0.108035,13.528144
3,3,0.220044,23.542791,0.006544,2,193.292901,0.005247,"risk, risk management, risk perception and cli...",0.001297,0.001297,...,0.220953,9,7,0.069191,0.184553,0.007645,0.050037,0.512424,0.174184,13.647592
4,4,0.143087,26.783303,0.007445,2,231.589446,0.006286,"humans, human activity, human behaviour & impa...",0.001159,0.001159,...,0.169152,5,7,0.065904,0.155102,0.017031,0.027815,0.621609,0.111205,13.911284


In [18]:
tdf[
    (tdf['Natural Sciences'] > tdf['Social Sciences']) & 
    (tdf['Natural Sciences'] > tdf['Agricultural Sciences']) & 
    (tdf['Natural Sciences'] > tdf['Engineering and Technology']) &
    (tdf['Natural Sciences'] > tdf['Humanities']) &
    (tdf['Natural Sciences'] > tdf['Medical and Health Sciences']) 
].shape

(115, 23)

In [43]:
from string import ascii_lowercase

def rep_time(ax, mdf, lquads=False, log_ticks=False, fs=7, quantiles=10):
    x = 'year_av'
    y = 'representation'
    y = 'lrep'
    
    mdf['lrep'] = np.log(mdf['representation'])

    cmap = {1: "#66c2a5", 2: "#fc8d62" , 3: "#8da0cb"}
    colors = [cmap[i] for i in mdf['primary_wg']]


    mdf.plot.scatter(
        x,y,s=mdf['score']*0.03,
        c = colors,ax=ax,
        lw=0.5,
        edgecolor="grey"
    )
    
    for k,v in cmap.items():
        ax.scatter(
            [],[],
            c=v,s=size,
            edgecolor="grey",
            lw=0.5,
            label=f"WG {k}",
        )   
        
    


    # Fit a trendline
    #z = np.polyfit(x=mdf[x], y=mdf[y], deg=1)
    #p = np.poly1d(z)
    #mdf['trendline'] = p(mdf.loc[:, x])
    #plt.plot(mdf[x],mdf['trendline'])


    # Quantile x and y
    qs = quantiles
    mdf['x_q'] = pd.qcut(mdf[x], qs, labels=False)
    mdf['y_q'] = pd.qcut(mdf[y], qs, labels=False)
    texts = []
    
    lfs = fs - 1
    if lfs < 5:
        lfs=5

    # write texts for dots in outer quantiles
    for i, row in mdf[(mdf['x_q'].isin([0,qs-1])) | (mdf['y_q'].isin([0,qs-1]))].iterrows(): 
        t = row['title'].split(',')[0].strip('{} ')
        texts.append(
            ax.text(row[x],row[y],t,ha='center', va='center',fontsize=lfs)
        )


    # Split the graph into quadrants
    ymid = 0
    ax.axhline(ymid,c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)
    ax.axvline(mdf[x].median(),c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)



    # Put labels on the quadrants
    # x
    d1 = mdf[x].median()-plt.xlim()[0]
    d2 = mdf[x].median()-plt.xlim()[1]

    xlabpoints = [mdf[x].median()-d1/2,mdf[x].median()-d2/2]
    
    xrange = mdf[x].max()-mdf[x].min()
    xlabpoints = [mdf[x].min()+xrange*0.15,mdf[x].min()+xrange*0.85]
    
    pad = (mdf[x].min() - ax.get_xlim()[0])*0.25
    
    xlabpoints = [ax.get_xlim()[0] + pad, mdf[x].median() + pad]
    

    
    if lquads:
        labels = [
            ['Under-represented,\nolder topics','Under-represented,\nnewer topics'],
            ['Over-represented,\nolder topics','Over-represented,\nnewer topics']
        ]

        pylims = ax.get_ylim()

        yr = pylims[1]-pylims[0]
        
        ax.set_ylim(pylims[0]-yr*0.05,pylims[1]+yr*0.05)
        
        pylims = ax.get_ylim()
        
        tpad = (pylims[1]-pylims[0])*0.13
        
        tpad=-pad*12

        ops = [operator.lt,operator.gt]

        for i in [0,1]:
            for j in [0,1]:
                if j==0:
                    tpadx=tpad*-1
                else:
                    tpadx=tpad*1


                q = mdf[(ops[i](mdf[x],mdf[x].median())) & (ops[j](mdf[y],ymid))]
                qshare = q['score'].sum()/mdf['score'].sum()
                l = labels[j][i].replace("\n"," ")
                ax.text(
                    xlabpoints[i],
                    pylims[j]+tpadx,
                    f"{l} {qshare:.0%}",
                    va="center",ha="left",
                    fontweight='bold',
                    #bbox={'facecolor':'red', 'alpha':qshare*2-0.1, 'pad':6},
                    fontsize=fs
                )

    if log_ticks:
        set_log_ticks(mdf, ax, -4)

    #ax.get_yaxis().set_visible(False)
    #ax.tick_params(axis=u'both', which=u'both',length=0)
    #ax2.tick_params(axis=u'both', which=u'both',length=0)
    ax.set_xlabel('Assessment period occurence')
    ax.set_ylabel('IPCC representation')
    
    # Tweak the labels to avoid overlap    
    adjust_text(
        texts, 
        arrowprops=dict(arrowstyle="-", color='black', lw=0.5)
    )


### Proportionality for other values of K
fig = plt.figure(figsize=(dsize,dsize*1))

size = 25
for i, run_id in enumerate([1794, 1795, 1800, 1804, 1810, 1811]):
    mdf = pd.read_csv(f'../tables/newness_representation_{run_id}.csv')

    print(mdf.shape[0])
    
    ax = fig.add_subplot(3,2,i+1)
    #ax.margins(0.1,0.1)

    rep_time(ax, mdf, lquads=True, log_ticks=True, fs=5, quantiles=16)
    
    ax.set_title(f"{ascii_lowercase[i]}) K = {mdf.shape[0]}",loc="left", fontweight="bold")
    
    #print(ax.margins())

ax.legend(bbox_to_anchor=(0,0.1), loc="lower left")
fig.tight_layout()

fig.patch.set_facecolor('white')   

plt.savefig(
    '../plots_pub/topic_rep_ks.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


80
90
100
110
120
130


In [20]:
df.columns

Index(['ipcc_coverage', 'ipcc_score', 'ipcc_share', 'primary_wg', 'score',
       'share', 'title', 'deviation', 'abs_md', 'representation', 'ys',
       'year_av', 'lrep', 'x_q', 'y_q', 'Agricultural Sciences',
       'Engineering and Technology', 'Humanities',
       'Medical and Health Sciences', 'Natural Sciences', 'Social Sciences',
       'H'],
      dtype='object')

In [75]:
def get_t_prop(row, prop):
    t = Topic.objects.get(title=row['title'],)
    return getattr(t,prop)


topic_df = df[['title','primary_wg','score','share','ipcc_share','representation','Agricultural Sciences',
       'Engineering and Technology', 'Humanities',
       'Medical and Health Sciences', 'Natural Sciences', 'Social Sciences']]

topic_df = topic_df.sort_values('share',ascending=False).reset_index(drop=True)
topic_df['top_words'] = topic_df.apply(get_t_prop, args=('top_words',), axis=1)
topic_df['topic_id'] = topic_df.apply(get_t_prop, args=('id',), axis=1)

topic_df = topic_df.rename(columns={"title": "short title, long title"})

topic_df

,"short title, long title",primary_wg,score,share,ipcc_share,representation,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences,top_words,topic_id
0,"emissions, emissions",3,2148.229392,0.058312,0.048400,0.830010,0.116713,0.356265,0.002950,0.005328,0.393381,0.123368,"[emission, greenhouse-gas, inventory, nox, red...",170414
1,"in/de-crease, increase/decrease",2,1638.818491,0.044485,0.051512,1.157960,0.123544,0.178093,0.002552,0.014110,0.628079,0.052397,"[increase, decrease, due, result, increased, l...",170428
2,"indices, indices, correlation and relationships",2,734.869401,0.019948,0.016558,0.830062,0.095918,0.180539,0.002125,0.013524,0.637389,0.069004,"[index, vulnerability, correlation, indicator,...",170408
3,"soil, soil moisture and organic matter",2,574.380258,0.015591,0.006251,0.400954,0.356283,0.072334,0.001048,0.001604,0.554405,0.014131,"[soil, moisture, content, microbial, organic, ...",170410
4,"areas, analysis of areas (including protected ...",2,439.024561,0.011917,0.009577,0.803672,0.098399,0.166888,0.004440,0.013838,0.651221,0.064058,"[area, study, high, map, coastal, protect, zon...",170536
5,"modelling, modelling & simulations",1,432.294256,0.011734,0.021865,1.863288,0.076132,0.201785,0.002164,0.007966,0.653724,0.057149,"[model, simulation, simulate, use, prediction,...",170418
6,"global, global scale observations/analysis",1,412.027572,0.011184,0.025289,2.261133,0.052898,0.159953,0.008065,0.014063,0.657719,0.105786,"[global, earth, world, atmospheric, estimate, ...",170547
7,"spatial, spatial & scale effects & analysis",1,407.295338,0.011056,0.014942,1.351494,0.071275,0.153154,0.003510,0.007375,0.707549,0.056405,"[scale, spatial, large, variability, temporal,...",170540
8,"projects, CDM and other projects",2,394.609823,0.010711,0.013609,1.270508,0.058591,0.279250,0.009080,0.008755,0.521150,0.121077,"[project, cdm, projection, future, development...",170544
9,"concentrations, atmospheric & plant concentrat...",1,387.272549,0.010512,0.011542,1.097977,0.121197,0.175983,0.001076,0.008186,0.675833,0.017134,"[concentration, doc, high, atmospheric, ppm, m...",170534


In [76]:
topic_df.to_csv('../tables/topics.csv')

1861

In [91]:
stat = RunStats.objects.get(pk=run_id)

docs = Doc.objects.filter(query=stat.query).values('id','title','PY','wosarticle__di','authors','UT__UT','wosarticle__so')

doc_df = pd.DataFrame.from_dict(list(docs))
doc_df = doc_df.rename(columns = {'id': 'doc_id','UT__UT':'wos_id','wosarticle__di':'DOI','wosarticle__so':'source'})
doc_df.head()

doc_df.to_csv('../')

,PY,wos_id,authors,doc_id,title,DOI,source
0,2015.0,WOS:000367398100005,"Jost, JT",71411,RESISTANCE TO CHANGE: A SOCIAL PSYCHOLOGICAL P...,None,SOCIAL RESEARCH
1,2015.0,WOS:000367141000037,"Zhuang, QL, Zhuang, QL, Zhuang, QL, Zhu, XD, Z...",70620,Influence of changes in wetland inundation ext...,10.1088/1748-9326/10/9/095009,ENVIRONMENTAL RESEARCH LETTERS
2,2015.0,WOS:000369638700022,"Ogundipe, FO, Jimoh, OD",426377,Life Cycle Assessment of Municipal Solid Waste...,None,INTERNATIONAL JOURNAL OF ENVIRONMENTAL RESEARCH
3,2015.0,WOS:000365433300031,"Ataei, A, Biglari, M, Nedaei, M, Assareh, E, C...",196030,Techno-Economic Feasibility Study of Autonomou...,10.1002/ep.12121,ENVIRONMENTAL PROGRESS & SUSTAINABLE ENERGY
4,2015.0,WOS:000365056100011,"Rezaeyan, A, Tabatabaei-Nejad, SA, Khodapanah,...",143617,A laboratory study on capillary sealing effici...,10.1016/j.marpetgeo.2015.07.022,MARINE AND PETROLEUM GEOLOGY


In [88]:
docs.filter(authors__isnull=False).count()

278963

In [90]:
doctopics = DocTopic.objects.filter(run_id=run_id)

dt_df = pd.DataFrame.from_dict(list(doctopics.values('doc_id','topic_id','score')))

dt_df.to_csv('../tables/doctopics.csv',index=False)

del dt_df
del doctopics